In [5]:
from dotenv import load_dotenv
load_dotenv() 
import os

from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai
import os



c:\Users\vamsi\Miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=768, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [8]:
import json
data = json.load(open("reviews.json"))
print(data)

{'reviews': [{'professor': 'Dr. Alice Johnson', 'subject': 'Computer Science', 'stars': 4, 'review': 'Dr. Johnson is very knowledgeable and provides clear explanations. The coursework is challenging but rewarding.'}, {'professor': 'Dr. Bob Smith', 'subject': 'Mathematics', 'stars': 5, 'review': 'Dr. Smith makes complex topics easy to understand and is always available for help. Highly recommended!'}, {'professor': 'Dr. Carol Davis', 'subject': 'Physics', 'stars': 3, 'review': 'Dr. Davis is a good lecturer, but the class can be quite demanding. Extra study is needed to keep up.'}, {'professor': 'Dr. David Wilson', 'subject': 'Chemistry', 'stars': 4, 'review': 'Dr. Wilson is enthusiastic and engaging. The labs are well-organized and provide a hands-on learning experience.'}, {'professor': 'Dr. Emily Brown', 'subject': 'Biology', 'stars': 5, 'review': 'Dr. Brown is passionate about the subject and makes learning enjoyable. Her exams are fair and well-structured.'}, {'professor': 'Dr. Fran

In [32]:
processed_data = []

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))





for review in data["reviews"]:
        embedding = genai.embed_content(
            model="models/text-embedding-004",
            content=str(review),
        )
        processed_data.append({
            "values": embedding['embedding'],
            "id": review["professor"],
            "metadata": {
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
        )
print(processed_data[0])

{'values': [0.02634067, 0.013287711, -0.052881934, 0.040315073, 0.023649799, 0.0152715435, 0.019565336, 0.021157375, 0.0069335676, 0.02087402, 0.016592577, 0.04583076, 0.037365578, -0.0039085075, -0.014221265, -0.014799807, 0.03208434, 0.031440333, -0.08660825, 0.05127053, -0.04974452, -0.04418412, 0.017020468, -0.022884589, 0.009784842, -0.010186997, 0.039900854, -0.029088382, 0.06894283, -0.04123809, 0.016897563, 0.0330394, -0.019810425, -0.07588808, -0.035672147, 0.052513096, -0.009113507, -0.017286662, 0.012973192, -0.058188904, -0.056938175, -0.020297997, -0.014873424, 0.030961527, -0.020016016, -0.012275609, 0.0014502892, 0.043206155, -0.011922882, 0.07783824, 0.028265238, 0.035824094, -0.049157012, 0.05060002, -0.014726517, -0.057791088, -0.028225651, -0.038659826, 0.02160293, -0.04106399, -0.026827985, -0.032670923, -0.039352823, -0.07987066, 0.042818446, -0.018978603, -0.066181876, -0.020112956, 0.032214854, -0.007659963, -0.0060609984, -0.0080979075, 0.0006024843, 0.028435996

In [33]:

# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)


In [34]:
print(index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}
